In [1]:
import os
import pickle
import sys
import warnings
import tensorflow as tf
import pandas as pd
import numpy as np

/home/angrypark/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from dataloader import load_data

In [5]:
train_data, val_data = load_data("data/movie_review_phase1")

In [6]:
from dataloader import Preprocessor